<a href="https://colab.research.google.com/github/Exion007/Colab/blob/main/sample_training2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Training

In [207]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import xgboost as xgb

%matplotlib inline

In [208]:
df = pd.read_csv("topratedmovies.csv")
df = df.iloc[:, 1:]
df.head()

,id,genre_ids,title,overview,popularity,release_date,vote_average,vote_count
0,238,"[18, 80]",The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",119.438,1972-03-14,8.7,18448
1,278,"[18, 80]",The Shawshank Redemption,Framed in the 1940s for the double murder of h...,90.415,1994-09-23,8.7,24376
2,240,"[18, 80]",The Godfather Part II,In the continuing saga of the Corleone crime f...,70.637,1974-12-20,8.6,11144
3,424,"[18, 36, 10752]",Schindler's List,The true story of how businessman Oskar Schind...,48.096,1993-12-15,8.6,14421
4,19404,"[35, 18, 10749]",Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",26.588,1995-10-20,8.6,4225


In [209]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            10000 non-null  int64  
 1   genre_ids     10000 non-null  object 
 2   title         10000 non-null  object 
 3   overview      9998 non-null   object 
 4   popularity    10000 non-null  float64
 5   release_date  10000 non-null  object 
 6   vote_average  10000 non-null  float64
 7   vote_count    10000 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 625.1+ KB


In [210]:
df.dropna(inplace = True)
df = df.loc[:, ['vote_average', 'popularity', 'vote_count']]

def to_list(row):
  row = row.replace(".", "")
  row = row[1:-1].split(", ")
  return row

def convert(row):
  return [int(item) for item in row]

#df['genre_ids'] = df['genre_ids'].apply(to_list)

df.head(3)

,vote_average,popularity,vote_count
0,8.7,119.438,18448
1,8.7,90.415,24376
2,8.6,70.637,11144


In [211]:
X = df.drop('vote_average', axis=1)
y = df['vote_average']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [212]:
model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train_scaled, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [213]:
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 0.6229695469451771


In [214]:
def vote_inputs():
  genre_ids = []
  x = True
  while (x):
    genre_id = int(input("Enter genre id: "))

    if (genre_id == -1):
      x = False

    else:
      if genre_id not in genre_ids:
        genre_ids.append(genre_id)
  return genre_ids

new_vote_count = int(input("Enter vote_count: "))
new_popularity = int(input("Enter popularity: "))
#new_genre_ids = vote_inputs()

new_row = pd.DataFrame({
    #'genre_ids' : [new_genre_ids],
    "popularity" : [new_popularity],
    "vote_count" : [new_vote_count]
})

new_row_scaled = scaler.transform(new_row)
predicted_rating = model.predict(new_row_scaled)[0]
print(f"Predicted Average Rating: {predicted_rating}")

Enter vote_count: 30000
Enter popularity: 1000
Predicted Average Rating: 8.211639404296875
